In [1]:
import os
os.chdir('..')
!pwd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/krishna/Documents/Research/Simultaneous Classifier/folktables-experiments/multidecomp


In [2]:
from bilevel.synth_datagen import SynthGenLinear
from bilevel.utils import *
from bilevel.OnlineLinearExpert import *
from bilevel.build_all_models import *
from bilevel.manual_inv_LinearExpert import Manual_inv_LinearExpert

In [3]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')
print(df_adult.shape)

numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']
cat_feat =  ['workclass', 'marital-status', 'relationship', 'native-country', 'occupation', 'race', 'sex']

df_adult.drop(['education'], axis = 1, inplace=True)
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

(49531, 14)


,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [4]:
# [col for col in df_adult_mm_oh.columns if '_' not in col]
X_dat = df_adult_mm_oh.drop('income', axis=1)
y_dat = df_adult_mm_oh['income']
print(X_dat.shape, y_dat.shape)

(49531, 91) (49531,)


## Online ridge by river has some issues, 
- the online least squares (river) still performs as well as batch LS(sklearn), 
- but online ridge (river) is far from batch ridge (sklearn)

In [33]:
# training the batch models
from sklearn.linear_model import Ridge, LinearRegression

ridge_sklearn = Ridge()
ridge_sklearn.fit(X_dat, y_dat)

ls_sklearn = LinearRegression()
ls_sklearn.fit(X_dat, y_dat)

ridge_sklearn.score(X_dat, y_dat), ls_sklearn.score(X_dat,y_dat)  # batch sklearn

(0.5803383358929834, 0.5801006608203434)

In [6]:
# training online models
ridge_river = OnlineLinearExpert(X_dat, y_dat, l2_pen=1.0)
T = y_dat.shape[0]
for t in tqdm(range(T)):
    ridge_river.get_ypred_t(t)
    ridge_river.update_t(t)

ls_river = OnlineLinearExpert(X_dat, y_dat, l2_pen=0.0)
T = y_dat.shape[0]
for t in tqdm(range(T)):
    ls_river.get_ypred_t(t)
    ls_river.update_t(t)

In [44]:
# comparing ridge results
from sklearn.metrics import r2_score

river_ridge_pred = ridge_river.model.predict_many(X_dat)
sklearn_ridge_pred = ridge_sklearn.predict(X_dat)

r2_score(y_dat, river_ridge_pred), r2_score(y_dat, ridge_river.y_predarr) ,r2_score(y_dat, sklearn_ridge_pred)  # the middle one is truly online


(0.29933857268425745, 0.3500720521473637, 0.5803383358929834)

In [45]:
# comparing ls results
river_ls_pred = ls_river.model.predict_many(X_dat)
sklearn_ls_pred = ls_sklearn.predict(X_dat)
r2_score(y_dat, river_ls_pred), r2_score(y_dat, ls_river.y_predarr), r2_score(y_dat, sklearn_ls_pred)  # the middle one is truly online, has clipped y's see the code

(0.5504581311141392, 0.5699849857449037, 0.5801006608203434)

### Testing SGD Least squares regressor

In [5]:
from sklearn.linear_model import SGDRegressor
mod_pac = SGDRegressor()
X_dat_np = X_dat.to_numpy()
y_dat_np = y_dat.to_numpy()
mod_pac.partial_fit([X_dat_np[0]], [y_dat_np[0]])
mod_pac.predict([X_dat_np[0]]), mod_pac.predict(X_dat_np[:1])


(array([0.04304694]), array([0.04304694]))

In [95]:
from sklearn.linear_model import SGDRegressor

mod_pac = SGDRegressor(warm_start = True, alpha = 0.01)
X_dat_np = X_dat.to_numpy()
y_dat_np = y_dat.to_numpy()
y_pred_sgd_ls = []
mod_pac.partial_fit([X_dat_np[0]], [y_dat_np[0]]) # just to start the init

for t in tqdm(range(len(y_dat_np))):
    # This is where we learn on a single datapoint
    y_pred_sgd_ls.append(np.clip(mod_pac.predict([X_dat_np[t]]), 0.0, 1.0))
    mod_pac.partial_fit([X_dat_np[t]], [y_dat_np[t]])
    

100%|██████████| 49531/49531 [00:24<00:00, 2039.42it/s]


In [96]:
r2_score(y_dat, y_pred_sgd_ls)

0.5251944090162931

In [97]:
np.linalg.norm(mod_pac.coef_)

0.4259548692713897

In [77]:
r2_score(y_dat, y_pred_sgd_ls) # using alpha = 1.0, is bad

0.2794660700249235

In [98]:
from sklearn.linear_model import Ridge

bo_ridge = Ridge() # batch version used online, l2 penalty = 1.0
X_dat_np = X_dat.to_numpy()
y_dat_np = y_dat.to_numpy()
y_pred_bo_ridge = []

bo_ridge.fit([X_dat_np[0]], [y_dat_np[0]]) # just to start the init

for t in tqdm(range(len(y_dat_np))):
    y_pred_bo_ridge.append(np.clip(bo_ridge.predict([X_dat_np[t]]), 0.0, 1.0))
    bo_ridge.fit(X_dat_np[:t+1], y_dat_np[:t+1]) 

100%|██████████| 49531/49531 [33:03<00:00, 24.97it/s]  


In [100]:
r2_score(y_dat, y_pred_bo_ridge)

0.5842470180396927

In [105]:
X_dat_np[0].shape

(91,)

In [119]:
(np.identity(91) @ X_dat_np[0]).shape

(91,)

In [118]:
np.dot(X_dat_np[0][:2], [1,1])

0.508946044171093

In [112]:
y_dat_np[0]

0.49045969647218995

In [115]:
X_dat_np[0].reshape(-1,1).shape

(91, 1)

In [6]:
manual_ridge = Manual_inv_LinearExpert(X_dat_np, y_dat_np, l2_pen = 1.0)

In [7]:
T = X_dat_np.shape[0]
for t in tqdm(range(T)):
    manual_ridge.get_ypred_t(t)
    manual_ridge.update_t(t)

100%|██████████| 49531/49531 [00:20<00:00, 2393.71it/s]


In [9]:
from sklearn.metrics import r2_score
r2_score(y_dat_np, manual_ridge.y_predarr)

0.5842159967472835